# dependencies
connect to cpu if want to use behaviour cloning

environment interaction numbers modified and aligned

rerun complete - 2024-03-12

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 22.2 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install seals
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823

# environment and expert

In [4]:
import gymnasium as gym
import imitation
import seals

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
SEED = 42

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:

def make_env_expert_rollouts(n_trajs):
    # Create the environment, load expert policy, and sample trajectories
    # from the expert policy and environment.
    global SEED
    env = make_vec_env(
        "seals/Walker2d-v1", # seals:seals/CartPole-v0
        rng=np.random.default_rng(SEED),
        n_envs=n_trajs,
        post_wrappers=[
            lambda env, _: RolloutInfoWrapper(env)
        ],  # needed for computing rollouts later
        parallel=True
    )
    expert = load_policy(
        "ppo-huggingface",
        organization="HumanCompatibleAI",
        env_name="seals/Walker2d-v1",
        venv=env,
    )
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=1000, min_episodes = n_trajs),
        rng=np.random.default_rng(SEED), #The random state to use for sampling trajectories
    )

    return env, expert, rollouts


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
import torch
def make_PPO_learner_and_trainer(env, expert, rollouts):
    global SEED

    learner_PPO = PPO(
        env=env,
        policy=MlpPolicy,
        batch_size=128,
        clip_range=0.4,
        ent_coef= 0.0007566389899529574,
        learning_rate=1e-4, #1.943992487657563e-5
        gae_lambda=0.92,
        max_grad_norm=0.6,
        gamma=0.98,
        n_epochs=20,
        seed=SEED,
    )

    load_from_saved_policy = torch.load("BC_MlpPolicy_Walker2D_1.pth.tar")
    learner_PPO.policy.load_state_dict(load_from_saved_policy.state_dict(), strict=False)

    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
    )
    gail_trainer = GAIL(
        demonstrations = rollouts, # expert trajectories
        demo_batch_size = 512,
        gen_replay_buffer_capacity = 16384,
        n_disc_updates_per_round = 16, #
        venv = env,
        gen_algo = learner_PPO,
        reward_net=reward_net,
        allow_variable_horizon=True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,

    )
    return learner_PPO, gail_trainer

# GAIL-PPO 1 trajectory - 1 traj BC

In [13]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)

In [14]:
learner_PPO_1, gail_trainer_PPO_1 = make_PPO_learner_and_trainer(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [15]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2481.58420425 +/- 210.94854271462816


In [16]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [17]:
gail_trainer_PPO_1.train(100000)
# An upper bound on the number of transitions to sample from the environment during training.

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 582      |
|    gen/time/fps             | 374      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.641    |
|    disc/disc_acc_expert             | 0.393    |
|    disc/disc_acc_gen                | 0.889    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.683    |
|    disc/disc_proportion_expert_pred | 0.252    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 512      |
|    disc/n_generated                 | 512      |
-

round:   2%|▏         | 1/48 [00:08<06:19,  8.08s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 684         |
|    gen/rollout/ep_rew_wrapped_mean | 714         |
|    gen/time/fps                    | 394         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 4096        |
|    gen/train/approx_kl             | 0.043824285 |
|    gen/train/clip_fraction         | 0.107       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -4.98       |
|    gen/train/explained_variance    | -0.782      |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 10.4        |
|    gen/train/n_updates             | 20          |
|    gen/train/policy_gradient_loss  | -0.0495     |
|    gen/train/std                   | 0.555  

round:   4%|▍         | 2/48 [00:14<05:34,  7.27s/it]

...

round:  96%|█████████▌| 46/48 [05:12<00:13,  6.86s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 1.59e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 653         |
|    gen/time/fps                    | 399         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 96256       |
|    gen/train/approx_kl             | 0.060589053 |
|    gen/train/clip_fraction         | 0.129       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -3.33       |
|    gen/train/explained_variance    | 0.645       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 1.57        |
|    gen/train/n_updates             | 920         |
|    gen/train/policy_gradient_loss  | -0.0688     |
|    gen/train/std                   | 0.421  

round:  98%|█████████▊| 47/48 [05:19<00:06,  6.80s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 1.6e+03     |
|    gen/rollout/ep_rew_wrapped_mean | 654         |
|    gen/time/fps                    | 387         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 98304       |
|    gen/train/approx_kl             | 0.054163035 |
|    gen/train/clip_fraction         | 0.139       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -3.3        |
|    gen/train/explained_variance    | 0.704       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 6.1         |
|    gen/train/n_updates             | 940         |
|    gen/train/policy_gradient_loss  | -0.0655     |
|    gen/train/std                   | 0.419  

round: 100%|██████████| 48/48 [05:26<00:00,  6.79s/it]


In [18]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [19]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 1796.2571327499998 +/- 515.7740038869783
Rewards after training: 2069.1033231499996 +/- 277.88218951030484


# GAIL-PPO 4 trajectory - 1 traj BC

In [20]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_PPO_4, gail_trainer_PPO_4 = make_PPO_learner_and_trainer(env_4, expert_4, rollouts_4)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [21]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2583.1990069999997 +/- 173.87006932840748


In [22]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [23]:
gail_trainer_PPO_4.train(100000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/12 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 819      |
|    gen/time/fps             | 1392     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 8192     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.612    |
|    disc/disc_acc_expert             | 0.346    |
|    disc/disc_acc_gen                | 0.879    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.686    |
|    disc/disc_proportion_expert_pred | 0.233    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 512      |
|    disc/n_generated                 | 512      |
-

round:   8%|▊         | 1/12 [00:11<02:04, 11.33s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 540         |
|    gen/rollout/ep_rew_wrapped_mean | 729         |
|    gen/time/fps                    | 1375        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 16384       |
|    gen/train/approx_kl             | 0.049863167 |
|    gen/train/clip_fraction         | 0.115       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -4.99       |
|    gen/train/explained_variance    | -0.801      |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.307       |
|    gen/train/n_updates             | 20          |
|    gen/train/policy_gradient_loss  | -0.0358     |
|    gen/train/std                   | 0.557  

round:  17%|█▋        | 2/12 [00:22<01:54, 11.40s/it]

...

round:  83%|████████▎ | 10/12 [01:54<00:22, 11.45s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 1.23e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 710         |
|    gen/time/fps                    | 1400        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 90112       |
|    gen/train/approx_kl             | 0.036363363 |
|    gen/train/clip_fraction         | 0.0816      |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -4.49       |
|    gen/train/explained_variance    | 0.947       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 2.26        |
|    gen/train/n_updates             | 200         |
|    gen/train/policy_gradient_loss  | -0.0305     |
|    gen/train/std                   | 0.508  

round:  92%|█████████▏| 11/12 [02:05<00:11, 11.38s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 1.28e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 705         |
|    gen/time/fps                    | 1374        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 98304       |
|    gen/train/approx_kl             | 0.033382878 |
|    gen/train/clip_fraction         | 0.0679      |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -4.41       |
|    gen/train/explained_variance    | 0.935       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 2.03        |
|    gen/train/n_updates             | 220         |
|    gen/train/policy_gradient_loss  | -0.0281     |
|    gen/train/std                   | 0.501  

round: 100%|██████████| 12/12 [02:16<00:00, 11.41s/it]


In [24]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [25]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 1953.8665905000003 +/- 417.2091002601491
learner rewards after training: 2023.9600819 +/- 397.99511091283114


# GAIL-PPO 10 trajectory - 1 traj BC

In [75]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_PPO_10, gail_trainer_PPO_10 = make_PPO_learner_and_trainer(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [76]:
print(rollouts_10.count)

<built-in method count of list object at 0x7b72810499c0>


In [77]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2551.0492796 +/- 203.9012591006724


In [78]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [79]:
gail_trainer_PPO_10.train(100000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/4 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 833      |
|    gen/time/fps             | 2697     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 7        |
|    gen/time/total_timesteps | 20480    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.624    |
|    disc/disc_acc_expert             | 0.354    |
|    disc/disc_acc_gen                | 0.895    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.685    |
|    disc/disc_proportion_expert_pred | 0.229    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 512      |
|    disc/n_generated                 | 512      |
-

round:  25%|██▌       | 1/4 [00:21<01:05, 21.84s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 649         |
|    gen/rollout/ep_rew_wrapped_mean | 726         |
|    gen/time/fps                    | 2707        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 7           |
|    gen/time/total_timesteps        | 40960       |
|    gen/train/approx_kl             | 0.039506685 |
|    gen/train/clip_fraction         | 0.102       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -4.95       |
|    gen/train/explained_variance    | -0.885      |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.135       |
|    gen/train/n_updates             | 20          |
|    gen/train/policy_gradient_loss  | -0.0238     |
|    gen/train/std                   | 0.551  

round:  50%|█████     | 2/4 [00:43<00:43, 21.94s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 502         |
|    gen/rollout/ep_rew_wrapped_mean | 683         |
|    gen/time/fps                    | 2729        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 7           |
|    gen/time/total_timesteps        | 61440       |
|    gen/train/approx_kl             | 0.037171613 |
|    gen/train/clip_fraction         | 0.0755      |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -4.91       |
|    gen/train/explained_variance    | 0.41        |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.0227      |
|    gen/train/n_updates             | 40          |
|    gen/train/policy_gradient_loss  | -0.0237     |
|    gen/train/std                   | 0.547  

round:  75%|███████▌  | 3/4 [01:05<00:21, 21.81s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 498         |
|    gen/rollout/ep_rew_wrapped_mean | 638         |
|    gen/time/fps                    | 2728        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 7           |
|    gen/time/total_timesteps        | 81920       |
|    gen/train/approx_kl             | 0.041488282 |
|    gen/train/clip_fraction         | 0.0856      |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | -4.84       |
|    gen/train/explained_variance    | 0.551       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.0213      |
|    gen/train/n_updates             | 60          |
|    gen/train/policy_gradient_loss  | -0.0237     |
|    gen/train/std                   | 0.538  

round: 100%|██████████| 4/4 [01:26<00:00, 21.75s/it]


In [80]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [81]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 2027.1987426500004 +/- 291.7824319188397
learner rewards after training: -34.81464625000001 +/- 474.64739214081266


# give trainer 10 traj BC network

In [54]:
def make_PPO_learner_and_trainer_10(env, expert, rollouts):
    global SEED

    learner_PPO = PPO(
        env=env,
        policy=MlpPolicy,
        batch_size=128,
        clip_range=0.4,
        ent_coef= 0.0007566389899529574,
        learning_rate=1e-4, #1.943992487657563e-5
        gae_lambda=0.92,
        max_grad_norm=0.6,
        gamma=0.98,
        n_epochs=20,
        seed=SEED,
    )

    load_from_saved_policy = torch.load("BC_MlpPolicy_Walker2D_10.pth.tar")
    learner_PPO.policy.load_state_dict(load_from_saved_policy.state_dict(), strict=False)

    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
    )
    gail_trainer = GAIL(
        demonstrations = rollouts, # expert trajectories
        demo_batch_size = 512,
        gen_replay_buffer_capacity = 16384,
        n_disc_updates_per_round = 16, #
        venv = env,
        gen_algo = learner_PPO,
        reward_net=reward_net,
        allow_variable_horizon=True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,

    )
    return learner_PPO, gail_trainer

# GAIL-PPO 1 trajectory - 10 traj BC

In [55]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)

In [56]:
learner_PPO_1, gail_trainer_PPO_1 = make_PPO_learner_and_trainer_10(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [57]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2481.58420425 +/- 210.94854271462816


In [58]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [59]:
gail_trainer_PPO_1.train(100000)
# An upper bound on the number of transitions to sample from the environment during training.

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 2.14e+03 |
|    gen/time/fps             | 369      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.562    |
|    disc/disc_acc_expert             | 0.336    |
|    disc/disc_acc_gen                | 0.789    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.691    |
|    disc/disc_proportion_expert_pred | 0.273    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 512      |
|    disc/n_generated                 | 512      |
-

round:   2%|▏         | 1/48 [00:07<05:37,  7.18s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 2.07e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 800         |
|    gen/time/fps                    | 361         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 4096        |
|    gen/train/approx_kl             | 0.054568373 |
|    gen/train/clip_fraction         | 0.159       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | 0.335       |
|    gen/train/explained_variance    | -0.773      |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 7.71        |
|    gen/train/n_updates             | 20          |
|    gen/train/policy_gradient_loss  | -0.0584     |
|    gen/train/std                   | 0.24   

round:   4%|▍         | 2/48 [00:14<05:33,  7.25s/it]

...

round:  96%|█████████▌| 46/48 [05:29<00:14,  7.17s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 2.24e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 601         |
|    gen/time/fps                    | 373         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 96256       |
|    gen/train/approx_kl             | 0.066008866 |
|    gen/train/clip_fraction         | 0.16        |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | 2.33        |
|    gen/train/explained_variance    | 0.593       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 3.33        |
|    gen/train/n_updates             | 920         |
|    gen/train/policy_gradient_loss  | -0.0775     |
|    gen/train/std                   | 0.175  

round:  98%|█████████▊| 47/48 [05:36<00:07,  7.16s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 2.24e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 600        |
|    gen/time/fps                    | 375        |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 5          |
|    gen/time/total_timesteps        | 98304      |
|    gen/train/approx_kl             | 0.07073817 |
|    gen/train/clip_fraction         | 0.176      |
|    gen/train/clip_range            | 0.4        |
|    gen/train/entropy_loss          | 2.36       |
|    gen/train/explained_variance    | 0.613      |
|    gen/train/learning_rate         | 0.0001     |
|    gen/train/loss                  | 2.64       |
|    gen/train/n_updates             | 940        |
|    gen/train/policy_gradient_loss  | -0.0752    |
|    gen/train/std                   | 0.174      |
|    gen/tra

round: 100%|██████████| 48/48 [05:44<00:00,  7.17s/it]


In [60]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [61]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 2483.9589683000004 +/- 282.340301557723
Rewards after training: 2398.8824824000003 +/- 216.62454331963792


# GAIL-PPO 4 trajectory - 10 traj BC

In [62]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_PPO_4, gail_trainer_PPO_4 = make_PPO_learner_and_trainer_10(env_4, expert_4, rollouts_4)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [63]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2583.1990069999997 +/- 173.87006932840748


In [64]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [65]:
gail_trainer_PPO_4.train(100000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/12 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 2.2e+03  |
|    gen/time/fps             | 1317     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 6        |
|    gen/time/total_timesteps | 8192     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.56     |
|    disc/disc_acc_expert             | 0.318    |
|    disc/disc_acc_gen                | 0.801    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.69     |
|    disc/disc_proportion_expert_pred | 0.259    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 512      |
|    disc/n_generated                 | 512      |
-

round:   8%|▊         | 1/12 [00:11<02:11, 11.96s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 2.21e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 802         |
|    gen/time/fps                    | 1312        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 6           |
|    gen/time/total_timesteps        | 16384       |
|    gen/train/approx_kl             | 0.054571494 |
|    gen/train/clip_fraction         | 0.142       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | 0.327       |
|    gen/train/explained_variance    | -0.684      |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.123       |
|    gen/train/n_updates             | 20          |
|    gen/train/policy_gradient_loss  | -0.0308     |
|    gen/train/std                   | 0.239  

round:  17%|█▋        | 2/12 [00:23<01:59, 11.91s/it]

...

round:  83%|████████▎ | 10/12 [01:59<00:23, 11.95s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 2.26e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 725        |
|    gen/time/fps                    | 1326       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 90112      |
|    gen/train/approx_kl             | 0.06412494 |
|    gen/train/clip_fraction         | 0.189      |
|    gen/train/clip_range            | 0.4        |
|    gen/train/entropy_loss          | 1.27       |
|    gen/train/explained_variance    | 0.385      |
|    gen/train/learning_rate         | 0.0001     |
|    gen/train/loss                  | 0.439      |
|    gen/train/n_updates             | 200        |
|    gen/train/policy_gradient_loss  | -0.0476    |
|    gen/train/std                   | 0.205      |
|    gen/tra

round:  92%|█████████▏| 11/12 [02:11<00:11, 11.97s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 2.26e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 722        |
|    gen/time/fps                    | 1313       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 98304      |
|    gen/train/approx_kl             | 0.06959724 |
|    gen/train/clip_fraction         | 0.181      |
|    gen/train/clip_range            | 0.4        |
|    gen/train/entropy_loss          | 1.41       |
|    gen/train/explained_variance    | 0.361      |
|    gen/train/learning_rate         | 0.0001     |
|    gen/train/loss                  | 0.409      |
|    gen/train/n_updates             | 220        |
|    gen/train/policy_gradient_loss  | -0.0496    |
|    gen/train/std                   | 0.201      |
|    gen/tra

round: 100%|██████████| 12/12 [02:23<00:00, 11.97s/it]


In [66]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [67]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 2426.61545595 +/- 238.26485089174446
learner rewards after training: 2378.25091935 +/- 220.85965367629078


# GAIL-PPO 10 trajectory - 10 traj BC

In [68]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_PPO_10, gail_trainer_PPO_10 = make_PPO_learner_and_trainer_10(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [69]:
print(rollouts_10.count)

<built-in method count of list object at 0x7b7280e70f00>


In [70]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2551.0492796 +/- 203.9012591006724


In [71]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [72]:
gail_trainer_PPO_10.train(100000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/4 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 2.22e+03 |
|    gen/time/fps             | 2657     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 7        |
|    gen/time/total_timesteps | 20480    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.531    |
|    disc/disc_acc_expert             | 0.314    |
|    disc/disc_acc_gen                | 0.748    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.691    |
|    disc/disc_proportion_expert_pred | 0.283    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 512      |
|    disc/n_generated                 | 512      |
-

round:  25%|██▌       | 1/4 [00:22<01:07, 22.45s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 2.23e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 804        |
|    gen/time/fps                    | 2678       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 7          |
|    gen/time/total_timesteps        | 40960      |
|    gen/train/approx_kl             | 0.04473298 |
|    gen/train/clip_fraction         | 0.114      |
|    gen/train/clip_range            | 0.4        |
|    gen/train/entropy_loss          | 0.363      |
|    gen/train/explained_variance    | -0.694     |
|    gen/train/learning_rate         | 0.0001     |
|    gen/train/loss                  | 0.243      |
|    gen/train/n_updates             | 20         |
|    gen/train/policy_gradient_loss  | -0.0173    |
|    gen/train/std                   | 0.237      |
|    gen/tra

round:  50%|█████     | 2/4 [00:44<00:44, 22.13s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 2.22e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 760         |
|    gen/time/fps                    | 2663        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 7           |
|    gen/time/total_timesteps        | 61440       |
|    gen/train/approx_kl             | 0.054745544 |
|    gen/train/clip_fraction         | 0.133       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | 0.413       |
|    gen/train/explained_variance    | 0.525       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.0541      |
|    gen/train/n_updates             | 40          |
|    gen/train/policy_gradient_loss  | -0.0172     |
|    gen/train/std                   | 0.236  

round:  75%|███████▌  | 3/4 [01:06<00:21, 21.94s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 2.23e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 756         |
|    gen/time/fps                    | 2735        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 7           |
|    gen/time/total_timesteps        | 81920       |
|    gen/train/approx_kl             | 0.055991895 |
|    gen/train/clip_fraction         | 0.148       |
|    gen/train/clip_range            | 0.4         |
|    gen/train/entropy_loss          | 0.489       |
|    gen/train/explained_variance    | 0.525       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.0853      |
|    gen/train/n_updates             | 60          |
|    gen/train/policy_gradient_loss  | -0.0236     |
|    gen/train/std                   | 0.232  

round: 100%|██████████| 4/4 [01:27<00:00, 21.97s/it]


In [73]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [74]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 2519.3324442 +/- 226.4117058487512
learner rewards after training: 2360.8957313 +/- 559.1727636737746
